In [1]:
import pandas as pd
import numpy as np
import re

##### AFTER SEARCHING IN GOOGLE, IN EU-27-LAU-2024-NUTS-2024 (EXCEL FILE FROM EU COMMISSION WITH NUTS 3 REGION CODES OF LOCAL ADMINISTRATIVE UNITS), AND CHECKED IN NUTS2021-NUTS2024 (EXCEL FILE FROM EU COMMISSION WITH IDENTIFYING CHANGES MADE IN CODES AND NAMES OF NUTS REGION IN EUROPE FROM 2021) I'LL MERGE THE NUTS 3 REGION CODES OF MY AOIS FROM THE EXCEL I CREATED (THAT HAS UNIQUE AOI NAMES) NAMED aois_with_nuts_regions TO AN EXCEL NAMED nuts3_regions_2024_typologies TO IDENTIFY DERIVED FROM A SHEET IN NUTS2021-NUTS2024 FILE, TO FIND IF THE AOIS/REGIONS ARE RURAL, URBAN, AND COASTAL. THEN I'LL MERGE WITH THE FINAL DATASET.


##### I'll merge them in final_all_aois_details where i have all the data from copernicus (reminder the max flood extent here and population is not yet correct here)

### final_all_aois_details is now clean in terms of AOIS (has only AOIs that do not overlap) 

In [3]:
df_cop = pd.read_excel("final_all_aois_details.xlsx")
df_cop

,activation_code,event_name,reason,activator,eventTime,activationTime,countries,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,reportLink,Unnamed: 19
0,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Buildings,hectares,"Built-up_School, university and research build...","schools, universities, and research buildings",public service and educational buildings,51.1,https://storymaps.arcgis.com/stories/9ff57e932...,NaN
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Other_(ha),unclassified land,other land,4573.0,https://storymaps.arcgis.com/stories/9ff57e932...,NaN
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Forests _(ha),forests,natural ecosystems,47.0,https://storymaps.arcgis.com/stories/9ff57e932...,NaN
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Pastures _(ha),pastures,agricultural land,150.7,https://storymaps.arcgis.com/stories/9ff57e932...,NaN
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Arable land_(ha),arable land,agricultural land,18462.8,https://storymaps.arcgis.com/stories/9ff57e932...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Secondary Road_(km),secondary roads,road infrastructure,3.0,NaN,NaN
1696,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Airfield runways_(km),airfield runways,aviation infrastructure,0.5,NaN,NaN
1697,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Long-distance railways_(km),long-distance railways,railway infrastructure,0.9,NaN,NaN
1698,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,NaN,number,Estimated population_None_(),NaN,NaN,2100.0,NaN,NaN


In [4]:
df_cop['aoiName'].nunique()

99

### the dataset below was created manually by serching the administrative regions of the AOIs

In [5]:
df = pd.read_excel("aois_with_nuts_regions.xlsx")
df

,aoiName (copernicus data),nuts_3_region_2024,nuts3_code_2021_2024
0,Valencia province,Valencia/València,ES523
1,Letur,Albacete,ES421
2,Castellon province,Castellón/Castelló,ES522
3,Reggio Emilia,Reggio nell'Emilia,ITH53
4,Modena,Modena,ITH54
...,...,...,...
93,Forlì,Forlì-Cesena,ITH58
94,Idice-Molinella,Bologna,ITH55
95,Lamone-Faenza,Ravenna,ITH57
96,Ravenna,Ravenna,ITH57


In [6]:
df['aoiName (copernicus data)'].nunique()

98

In [7]:
## i'll find the one aoi missing after i merge 

### this nuts3_regions_2024_typologies.xlsx is a dataset created manually from Eurostat's NUTS 2021- 2024 file to get typologies (urban-rural-coastal)

In [8]:
df_nuts_types = pd.read_excel("nuts3_regions_2024_typologies.xlsx")
df_nuts_types

,Country code,NUTS-3 Code,NUTS-3 label,Urban-Rural typology,Coastal regions,Metropolitan regions,Border regions,Island regions,Mountanious regions,Capital regions
0,BE,BE100,Arr. de Bruxelles-Capitale/Arr. van Brussel-Ho...,predominantly urban,non-coastal,NaN,NaN,NaN,NaN,Capital region
1,BE,BE211,Arr. Antwerpen,predominantly urban,coastal,NaN,NaN,NaN,NaN,NaN
2,BE,BE212,Arr. Mechelen,predominantly urban,coastal,NaN,NaN,NaN,NaN,NaN
3,BE,BE213,Arr. Turnhout,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
4,BE,BE223,Arr. Tongeren,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1160,SE,SE313,Gävleborgs län,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
1161,SE,SE321,Västernorrlands län,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
1162,SE,SE322,Jämtlands län,predominantly rural,non-coastal,NaN,NaN,NaN,NaN,NaN
1163,SE,SE331,Västerbottens län,intermediate,coastal,NaN,NaN,NaN,NaN,NaN


In [10]:
unique_df_nuts_types = df_nuts_types.drop_duplicates(subset=['NUTS-3 Code'], keep='first')

In [11]:
unique_df_nuts_types

,Country code,NUTS-3 Code,NUTS-3 label,Urban-Rural typology,Coastal regions,Metropolitan regions,Border regions,Island regions,Mountanious regions,Capital regions
0,BE,BE100,Arr. de Bruxelles-Capitale/Arr. van Brussel-Ho...,predominantly urban,non-coastal,NaN,NaN,NaN,NaN,Capital region
1,BE,BE211,Arr. Antwerpen,predominantly urban,coastal,NaN,NaN,NaN,NaN,NaN
2,BE,BE212,Arr. Mechelen,predominantly urban,coastal,NaN,NaN,NaN,NaN,NaN
3,BE,BE213,Arr. Turnhout,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
4,BE,BE223,Arr. Tongeren,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1160,SE,SE313,Gävleborgs län,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
1161,SE,SE321,Västernorrlands län,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
1162,SE,SE322,Jämtlands län,predominantly rural,non-coastal,NaN,NaN,NaN,NaN,NaN
1163,SE,SE331,Västerbottens län,intermediate,coastal,NaN,NaN,NaN,NaN,NaN


In [37]:
df_nuts_types_merge = pd.merge(df, df_nuts_types, left_on='nuts3_code_2021_2024', right_on='NUTS-3 Code', how='left')
df_nuts_types_merge

,aoiName (copernicus data),nuts_3_region_2024,nuts3_code_2021_2024,Country code,NUTS-3 Code,NUTS-3 label,Urban-Rural typology,Coastal regions,Metropolitan regions,Border regions,Island regions,Mountanious regions,Capital regions
0,Valencia province,Valencia/València,ES523,ES,ES523,Valencia/València,predominantly urban,coastal,NaN,NaN,NaN,NaN,NaN
1,Letur,Albacete,ES421,ES,ES421,Albacete,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
2,Castellon province,Castellón/Castelló,ES522,ES,ES522,Castellón/Castelló,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
3,Reggio Emilia,Reggio nell'Emilia,ITH53,IT,ITH53,Reggio nell’Emilia,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
4,Modena,Modena,ITH54,IT,ITH54,Modena,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,IT,ITH58,Forlì-Cesena,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
94,Idice-Molinella,Bologna,ITH55,IT,ITH55,Bologna,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN
95,Lamone-Faenza,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,NaN,NaN,NaN
96,Ravenna,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,NaN,NaN,NaN


#### create the columns for nuts 2 and nuts 1 regions to merge with NUTS2021-NUTS2024 data, the codes for each level are one character less from the previous NUTS 2 is NUTS 3 minus 1, NUTS 1 is NUTS 2 minus 1

In [38]:
df_nuts_types_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   aoiName (copernicus data)  98 non-null     object 
 1   nuts_3_region_2024         98 non-null     object 
 2   nuts3_code_2021_2024       78 non-null     object 
 3   Country code               78 non-null     object 
 4   NUTS-3 Code                78 non-null     object 
 5   NUTS-3 label               78 non-null     object 
 6   Urban-Rural typology       78 non-null     object 
 7   Coastal regions            78 non-null     object 
 8   Metropolitan regions       0 non-null      float64
 9   Border regions             0 non-null      float64
 10  Island regions             0 non-null      float64
 11  Mountanious regions        0 non-null      float64
 12  Capital regions            1 non-null      object 
dtypes: float64(4), object(9)
memory usage: 10.1+ KB


In [39]:
df_nuts_types_merge['nuts2_code_2021_2024'] = df_nuts_types_merge['nuts3_code_2021_2024'].astype(str).apply(lambda x: x[:-1] if x else x)
df_nuts_types_merge['nuts1_code_2021_2024'] = df_nuts_types_merge['nuts3_code_2021_2024'].astype(str).apply(lambda x: x[:-2] if x else x)

In [41]:
df_nuts_types_merge

,aoiName (copernicus data),nuts_3_region_2024,nuts3_code_2021_2024,Country code,NUTS-3 Code,NUTS-3 label,Urban-Rural typology,Coastal regions,Metropolitan regions,Border regions,Island regions,Mountanious regions,Capital regions,nuts2_code_2021_2024,nuts1_code_2021_2024
0,Valencia province,Valencia/València,ES523,ES,ES523,Valencia/València,predominantly urban,coastal,NaN,NaN,NaN,NaN,NaN,ES52,ES5
1,Letur,Albacete,ES421,ES,ES421,Albacete,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN,ES42,ES4
2,Castellon province,Castellón/Castelló,ES522,ES,ES522,Castellón/Castelló,intermediate,coastal,NaN,NaN,NaN,NaN,NaN,ES52,ES5
3,Reggio Emilia,Reggio nell'Emilia,ITH53,IT,ITH53,Reggio nell’Emilia,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN,ITH5,ITH
4,Modena,Modena,ITH54,IT,ITH54,Modena,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN,ITH5,ITH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,IT,ITH58,Forlì-Cesena,intermediate,coastal,NaN,NaN,NaN,NaN,NaN,ITH5,ITH
94,Idice-Molinella,Bologna,ITH55,IT,ITH55,Bologna,intermediate,non-coastal,NaN,NaN,NaN,NaN,NaN,ITH5,ITH
95,Lamone-Faenza,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,NaN,NaN,NaN,ITH5,ITH
96,Ravenna,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,NaN,NaN,NaN,ITH5,ITH


In [42]:
df_nuts_types_merge.to_excel('aois_with_nuts_region_codes_and_types.xlsx', index=False)

#### now merge with NUTS2021-NUTS2024 first sheet that I put in another excel called all_nuts_2024_sheet_from_NUTS2021-NUTS2024

In [43]:
df_all_nuts = pd.read_excel("all_nuts_2024_sheet_from_NUTS2021-NUTS2024.xlsx")
df_all_nuts

,Country code,NUTS Code,NUTS label,NUTS level,Country order,#
0,BE,BE1,Région de Bruxelles-Capitale/Brussels Hoofdste...,1,1,1
1,BE,BE10,Région de Bruxelles-Capitale/Brussels Hoofdste...,2,1,2
2,BE,BE100,Arr. de Bruxelles-Capitale/Arr. Brussel-Hoofdstad,3,1,3
3,BE,BE2,Vlaams Gewest,1,1,4
4,BE,BE21,Prov. Antwerpen,2,1,5
...,...,...,...,...,...,...
1577,SE,SE331,Västerbottens län,3,27,1578
1578,SE,SE332,Norrbottens län,3,27,1579
1579,SE,SEZ,Extra-Regio NUTS 1,1,27,1580
1580,SE,SEZZ,Extra-Regio NUTS 2,2,27,1581


In [44]:
df_nuts_types_merge_nuts2 = pd.merge(df_nuts_types_merge, df_all_nuts, left_on='nuts2_code_2021_2024', right_on='NUTS Code', how='left')
df_nuts_types_merge_nuts2

,aoiName (copernicus data),nuts_3_region_2024,nuts3_code_2021_2024,Country code_x,NUTS-3 Code,NUTS-3 label,Urban-Rural typology,Coastal regions,Metropolitan regions,Border regions,...,Mountanious regions,Capital regions,nuts2_code_2021_2024,nuts1_code_2021_2024,Country code_y,NUTS Code,NUTS label,NUTS level,Country order,#
0,Valencia province,Valencia/València,ES523,ES,ES523,Valencia/València,predominantly urban,coastal,NaN,NaN,...,NaN,NaN,ES52,ES5,ES,ES52,Comunitat Valenciana,2.0,9.0,753.0
1,Letur,Albacete,ES421,ES,ES421,Albacete,intermediate,non-coastal,NaN,NaN,...,NaN,NaN,ES42,ES4,ES,ES42,Castilla-La Mancha,2.0,9.0,738.0
2,Castellon province,Castellón/Castelló,ES522,ES,ES522,Castellón/Castelló,intermediate,coastal,NaN,NaN,...,NaN,NaN,ES52,ES5,ES,ES52,Comunitat Valenciana,2.0,9.0,753.0
3,Reggio Emilia,Reggio nell'Emilia,ITH53,IT,ITH53,Reggio nell’Emilia,intermediate,non-coastal,NaN,NaN,...,NaN,NaN,ITH5,ITH,IT,ITH5,Emilia-Romagna,2.0,12.0,1059.0
4,Modena,Modena,ITH54,IT,ITH54,Modena,intermediate,non-coastal,NaN,NaN,...,NaN,NaN,ITH5,ITH,IT,ITH5,Emilia-Romagna,2.0,12.0,1059.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,IT,ITH58,Forlì-Cesena,intermediate,coastal,NaN,NaN,...,NaN,NaN,ITH5,ITH,IT,ITH5,Emilia-Romagna,2.0,12.0,1059.0
94,Idice-Molinella,Bologna,ITH55,IT,ITH55,Bologna,intermediate,non-coastal,NaN,NaN,...,NaN,NaN,ITH5,ITH,IT,ITH5,Emilia-Romagna,2.0,12.0,1059.0
95,Lamone-Faenza,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,...,NaN,NaN,ITH5,ITH,IT,ITH5,Emilia-Romagna,2.0,12.0,1059.0
96,Ravenna,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,...,NaN,NaN,ITH5,ITH,IT,ITH5,Emilia-Romagna,2.0,12.0,1059.0


In [45]:
df_nuts_types_merge_nuts1 = pd.merge(df_nuts_types_merge_nuts2, df_all_nuts, left_on='nuts1_code_2021_2024', right_on='NUTS Code', how='left')
df_nuts_types_merge_nuts1

,aoiName (copernicus data),nuts_3_region_2024,nuts3_code_2021_2024,Country code_x,NUTS-3 Code,NUTS-3 label,Urban-Rural typology,Coastal regions,Metropolitan regions,Border regions,...,NUTS label_x,NUTS level_x,Country order_x,#_x,Country code,NUTS Code_y,NUTS label_y,NUTS level_y,Country order_y,#_y
0,Valencia province,Valencia/València,ES523,ES,ES523,Valencia/València,predominantly urban,coastal,NaN,NaN,...,Comunitat Valenciana,2.0,9.0,753.0,ES,ES5,Este,1.0,9.0,747.0
1,Letur,Albacete,ES421,ES,ES421,Albacete,intermediate,non-coastal,NaN,NaN,...,Castilla-La Mancha,2.0,9.0,738.0,ES,ES4,Centro (ES),1.0,9.0,727.0
2,Castellon province,Castellón/Castelló,ES522,ES,ES522,Castellón/Castelló,intermediate,coastal,NaN,NaN,...,Comunitat Valenciana,2.0,9.0,753.0,ES,ES5,Este,1.0,9.0,747.0
3,Reggio Emilia,Reggio nell'Emilia,ITH53,IT,ITH53,Reggio nell’Emilia,intermediate,non-coastal,NaN,NaN,...,Emilia-Romagna,2.0,12.0,1059.0,IT,ITH,Nord-Est,1.0,12.0,1041.0
4,Modena,Modena,ITH54,IT,ITH54,Modena,intermediate,non-coastal,NaN,NaN,...,Emilia-Romagna,2.0,12.0,1059.0,IT,ITH,Nord-Est,1.0,12.0,1041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,IT,ITH58,Forlì-Cesena,intermediate,coastal,NaN,NaN,...,Emilia-Romagna,2.0,12.0,1059.0,IT,ITH,Nord-Est,1.0,12.0,1041.0
94,Idice-Molinella,Bologna,ITH55,IT,ITH55,Bologna,intermediate,non-coastal,NaN,NaN,...,Emilia-Romagna,2.0,12.0,1059.0,IT,ITH,Nord-Est,1.0,12.0,1041.0
95,Lamone-Faenza,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,...,Emilia-Romagna,2.0,12.0,1059.0,IT,ITH,Nord-Est,1.0,12.0,1041.0
96,Ravenna,Ravenna,ITH57,IT,ITH57,Ravenna,intermediate,coastal,NaN,NaN,...,Emilia-Romagna,2.0,12.0,1059.0,IT,ITH,Nord-Est,1.0,12.0,1041.0


##### download it, input the nuts2 and nuts1 region names and codes missing since the aois where not in any nuts3 region specifically, and take out the columns you don't want manually, fix column names, then read again as clean

In [47]:
df_nuts_types_merge_nuts1.to_excel('aois_with_ALL_nuts_region_codes_and_types.xlsx', index=False)

## Try to translate the regions automatically

In [63]:
df_nuts_clean = pd.read_excel("aois_with_ALL_nuts_region_codes_and_types_clean.xlsx")
df_nuts_clean

,aoiName,nuts3_name_2024,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024
0,Valencia province,Valencia/València,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este
1,Letur,Albacete,ES421,intermediate,non-coastal,ES42,Castilla-La Mancha,ES4,Centro (ES)
2,Castellon province,Castellón/Castelló,ES522,intermediate,coastal,ES52,Comunitat Valenciana,ES5,Este
3,Reggio Emilia,Reggio nell'Emilia,ITH53,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est
4,Modena,Modena,ITH54,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est
...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est
94,Idice-Molinella,Bologna,ITH55,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est
95,Lamone-Faenza,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est
96,Ravenna,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est


In [59]:
!pip install deep-translator

   ------------------------- 0.0/42.3 kB ? eta -:--:--
   ----------------- 42.3/42.3 kB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
from deep_translator import GoogleTranslator

In [67]:
df_nuts_clean_translated_test = df_nuts_clean.copy()

df_nuts_clean_translated_test['nuts3_name_2024_en'] = df_nuts_clean['nuts3_name_2024'].astype(str).apply(
    lambda x: GoogleTranslator(source='auto', target='en').translate(x) if pd.notnull(x) else x
)
df_nuts_clean_translated_test['nuts2_name_2024_en'] = df_nuts_clean['nuts2_name_2024'].astype(str).apply(
    lambda x: GoogleTranslator(source='auto', target='en').translate(x) if pd.notnull(x) else x
)
df_nuts_clean_translated_test['nuts1_name_2024_en'] = df_nuts_clean['nuts1_name_2024'].astype(str).apply(
    lambda x: GoogleTranslator(source='auto', target='en').translate(x) if pd.notnull(x) else x
)

In [68]:
df_nuts_clean_translated_test

,aoiName,nuts3_name_2024,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024,nuts3_name_2024_en,nuts2_name_2024_en,nuts1_name_2024_en
0,Valencia province,Valencia/València,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia/Valencia,Valencian Community,It is
1,Letur,Albacete,ES421,intermediate,non-coastal,ES42,Castilla-La Mancha,ES4,Centro (ES),Albacete,Castilla-La Mancha,Center (s)
2,Castellon province,Castellón/Castelló,ES522,intermediate,coastal,ES52,Comunitat Valenciana,ES5,Este,Castellón/Castelló,Valencian Community,It is
3,Reggio Emilia,Reggio nell'Emilia,ITH53,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Reggio nel Emilia,Emilia-Romagna,Northeast
4,Modena,Modena,ITH54,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Modena,Emilia-Romagna,Northeast
...,...,...,...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena,Emilia-Romagna,Northeast
94,Idice-Molinella,Bologna,ITH55,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Bologna,Error 500 (Server Error)!!1500.That’s an error...,Northeast
95,Lamone-Faenza,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Ravenna,Emilia-Romagna,Northeast
96,Ravenna,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Ravenna,Emilia-Romagna,Northeast


In [69]:
df_nuts_clean_translated_test.to_excel("df_nuts_clean_translated_test.xlsx", index=False)

### Now merge your clean nuts region file with final copernicus dataset

In [70]:
df_cop = pd.read_excel("final_all_aois_details.xlsx")
df_cop

,activation_code,event_name,reason,activator,eventTime,activationTime,countries,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,reportLink
0,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Buildings,hectares,"Built-up_School, university and research build...","schools, universities, and research buildings",public service and educational buildings,51.1,https://storymaps.arcgis.com/stories/9ff57e932...
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Other_(ha),unclassified land,other land,4573.0,https://storymaps.arcgis.com/stories/9ff57e932...
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Forests _(ha),forests,natural ecosystems,47.0,https://storymaps.arcgis.com/stories/9ff57e932...
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Pastures _(ha),pastures,agricultural land,150.7,https://storymaps.arcgis.com/stories/9ff57e932...
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Arable land_(ha),arable land,agricultural land,18462.8,https://storymaps.arcgis.com/stories/9ff57e932...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Secondary Road_(km),secondary roads,road infrastructure,3.0,NaN
1696,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Airfield runways_(km),airfield runways,aviation infrastructure,0.5,NaN
1697,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Long-distance railways_(km),long-distance railways,railway infrastructure,0.9,NaN
1698,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,1.0,2023-05-19T13:30:00,NaN,number,Estimated population_None_(),NaN,NaN,2100.0,NaN


In [71]:
df_nuts_for_merge = pd.read_excel("df_nuts_clean_translated_test.xlsx")
df_nuts_for_merge

,aoiName,nuts3_name_2024,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024,nuts3_name_2024_en,nuts2_name_2024_en,nuts1_name_2024_en
0,Valencia province,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
1,Letur,Albacete,ES421,intermediate,non-coastal,ES42,Castilla-La Mancha,ES4,Centro (ES),Albacete province,Castilla-La Mancha region,Central Spain
2,Castellon province,Castellón,ES522,intermediate,coastal,ES52,Comunitat Valenciana,ES5,Este,Castellón province,Valencian Community,Eastern Spain
3,Reggio Emilia,Reggio nell'Emilia,ITH53,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Reggio Emilia province,Emilia-Romagna region,North-Eastern Italy
4,Modena,Modena,ITH54,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Modena province,Emilia-Romagna region,North-Eastern Italy
...,...,...,...,...,...,...,...,...,...,...,...,...
93,Forlì,Forlì-Cesena,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena province,Emilia-Romagna,North-Eastern Italy
94,Idice-Molinella,Bologna,ITH55,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Bologna province,Emilia-Romagna,North-Eastern Italy
95,Lamone-Faenza,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Ravenna province,Emilia-Romagna,North-Eastern Italy
96,Ravenna,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Ravenna province,Emilia-Romagna,North-Eastern Italy


In [72]:
df_cop_nuts_merge = pd.merge(df_cop, df_nuts_for_merge, on='aoiName', how='left')
df_cop_nuts_merge

,activation_code,event_name,reason,activator,eventTime,activationTime,countries,aoiName,aoiNumber,type,...,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024,nuts3_name_2024_en,nuts2_name_2024_en,nuts1_name_2024_en
0,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,...,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,...,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,...,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,...,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1.0,DEL,...,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,...,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena province,Emilia-Romagna,North-Eastern Italy
1696,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,...,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena province,Emilia-Romagna,North-Eastern Italy
1697,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,...,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena province,Emilia-Romagna,North-Eastern Italy
1698,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1.0,DEL,...,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena province,Emilia-Romagna,North-Eastern Italy


In [73]:
df_cop_nuts_merge.to_excel("final_all_aois_details_WITH_NUTS_TRANSLATED.xlsx", index=False)

### also fix population from the third time you scraped since copernicus changed the website to make sure the data for population are correct

In [76]:
df_scraped_data_new = pd.read_excel("new_aois_df_2 - Copy.xlsx")
df_scraped_data_new

,name,number,activationCode,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,subcategory_unit,affected,total,observed_event,maximum_flood_extent,maximum_water_extent
0,Olden,1.0,EMSR775,EMSR775,Olden,1.0,DEL,0.0,2024-11-01T14:12:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Oyane,2.0,EMSR775,EMSR775,Oyane,2.0,DEL,0.0,2024-11-01T18:30:00,NaN,NaN,NaN,NaN,NaN,NaN
2,Oyane,2.0,EMSR775,EMSR775,Oyane,2.0,DEL,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aurlandsvangen,3.0,EMSR775,EMSR775,Aurlandsvangen,3.0,DEL,0.0,2024-11-01T17:25:00,NaN,NaN,NaN,NaN,NaN,NaN
4,Over Ardal,4.0,EMSR775,EMSR775,Over Ardal,4.0,DEL,0.0,2024-11-01T20:43:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,5.0,2023-04-06T18:59:00,Transportation_Airfield runways_(km),0.9,12.4,NaN,NaN,NaN
7668,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,5.0,2023-04-06T18:59:00,Transportation_Long-distance railways_(km),0.5,316.6,NaN,NaN,NaN
7669,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,5.0,2023-04-06T18:59:00,Estimated population_None_(),300.0,83000.0,NaN,NaN,NaN
7670,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,5.0,2023-04-06T18:59:00,Previous flooded area_None_(ha),5485.1,5485.1,no,NaN,NaN


In [79]:
pd.set_option('display.max_columns', None)

In [99]:
df_cop_nuts_merge_clean = pd.read_excel("final_all_aois_details_WITH_NUTS_TRANSLATED.xlsx")
df_cop_nuts_merge_clean

,activation_code,event_name,reason,activator,eventTime,activationTime,countries,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,reportLink,nuts3_name_2024,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024,nuts3_name_2024_en,nuts2_name_2024_en,nuts1_name_2024_en
0,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Buildings,hectares,"Built-up_School, university and research build...","schools, universities, and research buildings",public service and educational buildings,51.1,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Other_(ha),unclassified land,other land,4573.0,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Forests _(ha),forests,natural ecosystems,47.0,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
3,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Pastures _(ha),pastures,agricultural land,150.7,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
4,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Land use,hectares,Land use_Arable land_(ha),arable land,agricultural land,18462.8,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1,DEL,1,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Secondary Road_(km),secondary roads,road infrastructure,3.0,NaN,Forlì-Cesena,ITH58,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Forlì-Cesena province,Emilia-Romagna,North-Eastern Italy
1696,EMSR664,Flood in Italy,A new wave of severe weather has again hit sou...,Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-16T11:00:00,2023-05-16T14:33:00,Italy,Forlì,1,DEL,1,2023-05-19T13:30:00,Transportation infrastructure,kilometres,Transportation_Airfield runways_(km),airfield runways,aviation infrastructure,0.5,NaN,Forlì-Cesena,ITH58,intermediate,coas

In [100]:
df_pop = df_cop_nuts_merge_clean[df_cop_nuts_merge_clean['category_unit'] == 'Population']
df_pop

,activation_code,event_name,reason,activator,eventTime,activationTime,countries,aoiName,aoiNumber,type,monitoring,deliveryTime,category_unit,unit_measurement,subcategory_unit,normalized_category_v1,normalized_category_v2,affected,reportLink,nuts3_name_2024,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024,nuts3_name_2024_en,nuts2_name_2024_en,nuts1_name_2024_en
29,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Population,number,population_affected_max,NaN,max population,190000.0,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain
44,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Letur,5,GRA,0,2024-11-07T20:41:36.273643,Population,number,population_affected_max,NaN,max population,10.0,https://storymaps.arcgis.com/stories/9ff57e932...,Albacete,ES421,intermediate,non-coastal,ES42,Castilla-La Mancha,ES4,Centro (ES),Albacete province,Castilla-La Mancha region,Central Spain
60,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Castellon province,6,DEL,1,2024-11-08T22:00:30.496128,Population,number,population_affected_max,NaN,max population,80.0,https://storymaps.arcgis.com/stories/9ff57e932...,Castellón,ES522,intermediate,coastal,ES52,Comunitat Valenciana,ES5,Este,Castellón province,Valencian Community,Eastern Spain
78,EMSR771,"Flood in Emilia- Romagna, Italy",Italy has been hit in recent days by a severe ...,Italy|Presidency of the Council of Ministers -...,2024-10-20T12:00:00,2024-10-20T08:24:00,Italy,Reggio Emilia,6,DEL,0,2024-10-21T15:43:26.721092,Population,number,population_affected_max,NaN,max population,150.0,https://storymaps.arcgis.com/stories/b95e2210e...,Reggio nell'Emilia,ITH53,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Reggio Emilia province,Emilia-Romagna region,North-Eastern Italy
96,EMSR771,"Flood in Emilia- Romagna, Italy",Italy has been hit in recent days by a severe ...,Italy|Presidency of the Council of Ministers -...,2024-10-20T12:00:00,2024-10-20T08:24:00,Italy,Modena,7,DEL,0,2024-10-22T16:42:04.899238,Population,number,population_affected_max,NaN,max population,100.0,https://storymaps.arcgis.com/stories/b95e2210e...,Modena,ITH54,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Modena province,Emilia-Romagna region,North-Eastern Italy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,Italy,Spazzate-Sassatelli,1,DEL,0,2023-05-08T20:13:00,Population,number,population_affected_max,NaN,max population,200.0,NaN,Bologna,ITH55,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Bologna province,Emilia-Romagna,North-Eastern Italy
1626,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,Italy,Lamone-Faenza,2,DEL,0,2023-05-08T20:14:00,Population,number,population_affected_max,NaN,max population,400.0,NaN,Ravenna,ITH57,intermediate,coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Ravenna province,Emilia-Romagna,North-Eastern Italy
1637,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy

In [84]:
df_scraped_data_new_pop = df_scraped_data_new[
    (df_scraped_data_new['subcategory_unit'] == 'Estimated population_None_()') |
    (df_scraped_data_new['subcategory_unit'] == 'Estimated population__(nan)')
]
df_scraped_data_new_pop

,name,number,activationCode,activation_code,aoiName,aoiNumber,type,monitoring,deliveryTime,subcategory_unit,affected,total,observed_event,maximum_flood_extent,maximum_water_extent
14,Viksdalen,11.0,EMSR775,EMSR775,Viksdalen,11.0,DEL,0.0,2024-11-05T19:18:42.042642,Estimated population_None_(),100.0,100.0,NaN,NaN,NaN
38,Valencia province,1.0,EMSR773,EMSR773,Valencia province,1.0,DEL,4.0,2024-11-08T10:55:45.498870,Estimated population_None_(),550.0,2.5,NaN,NaN,NaN
68,Valencia province,1.0,EMSR773,EMSR773,Valencia province,1.0,DEL,2.0,2024-11-06T07:08:12.297967,Estimated population_None_(),3600.0,2.5,NaN,NaN,NaN
89,Valencia province,1.0,EMSR773,EMSR773,Valencia province,1.0,DEL,3.0,2024-11-06T13:10:56.058275,Estimated population_None_(),750.0,2.5,NaN,NaN,NaN
127,Valencia province,1.0,EMSR773,EMSR773,Valencia province,1.0,DEL,0.0,2024-11-02T03:02:32.084063,Estimated population_None_(),190000.0,2.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7586,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,0.0,2023-04-02T06:33:00,Estimated population_None_(),100.0,83000.0,NaN,NaN,NaN
7607,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,1.0,2023-04-03T01:21:00,Estimated population_None_(),100.0,83000.0,NaN,NaN,NaN
7629,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,6.0,2023-04-08T23:45:00,Estimated population_None_(),400.0,83000.0,NaN,NaN,NaN
7648,DaugavaRiver,1.0,EMSR657,EMSR657,DaugavaRiver,1.0,DEL,4.0,2023-04-05T15:59:00,Estimated population_None_(),150.0,83000.0,NaN,NaN,NaN


In [101]:
merged_df = df_pop.merge(df_scraped_data_new_pop, 
                      on=['aoiName', 'aoiNumber', 'type', 'monitoring'], 
                      how='left') 

In [102]:
merged_df

,activation_code_x,event_name,reason,activator,eventTime,activationTime,countries,aoiName,aoiNumber,type,monitoring,deliveryTime_x,category_unit,unit_measurement,subcategory_unit_x,normalized_category_v1,normalized_category_v2,affected_x,reportLink,nuts3_name_2024,nuts3_code_2024,nuts3_2024_urban_rural_typology,nuts3_2024_coastal_regions,nuts2_code_2024,nuts2_name_2024,nuts1_code_2024,nuts1_name_2024,nuts3_name_2024_en,nuts2_name_2024_en,nuts1_name_2024_en,name,number,activationCode,activation_code_y,deliveryTime_y,subcategory_unit_y,affected_y,total,observed_event,maximum_flood_extent,maximum_water_extent
0,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Valencia province,1,DEL,0,2024-11-02T03:02:32.084063,Population,number,population_affected_max,NaN,max population,190000.0,https://storymaps.arcgis.com/stories/9ff57e932...,Valencia,ES523,predominantly urban,coastal,ES52,Comunitat Valenciana,ES5,Este,Valencia province,Valencian Community,Eastern Spain,Valencia province,1.0,EMSR773,EMSR773,2024-11-02T03:02:32.084063,Estimated population_None_(),190000.0,2.5,NaN,NaN,NaN
1,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Letur,5,GRA,0,2024-11-07T20:41:36.273643,Population,number,population_affected_max,NaN,max population,10.0,https://storymaps.arcgis.com/stories/9ff57e932...,Albacete,ES421,intermediate,non-coastal,ES42,Castilla-La Mancha,ES4,Centro (ES),Albacete province,Castilla-La Mancha region,Central Spain,Letur,5.0,EMSR773,EMSR773,2024-11-07T20:41:36.273643,Estimated population_None_(),10.0,500.0,NaN,NaN,NaN
2,EMSR773,"Flood in Valencia Region, Spain","On 29 October 2024 at 14:30 UTC, an extraordin...",Spain|Ministry of Interior - Centro Nacional d...,2024-10-29T14:30:00,2024-10-29T16:40:00,Spain,Castellon province,6,DEL,1,2024-11-08T22:00:30.496128,Population,number,population_affected_max,NaN,max population,80.0,https://storymaps.arcgis.com/stories/9ff57e932...,Castellón,ES522,intermediate,coastal,ES52,Comunitat Valenciana,ES5,Este,Castellón province,Valencian Community,Eastern Spain,Castellon province,6.0,EMSR773,EMSR773,2024-11-08T22:00:30.496128,Estimated population_None_(),80.0,70000.0,NaN,NaN,NaN
3,EMSR771,"Flood in Emilia- Romagna, Italy",Italy has been hit in recent days by a severe ...,Italy|Presidency of the Council of Ministers -...,2024-10-20T12:00:00,2024-10-20T08:24:00,Italy,Reggio Emilia,6,DEL,0,2024-10-21T15:43:26.721092,Population,number,population_affected_max,NaN,max population,150.0,https://storymaps.arcgis.com/stories/b95e2210e...,Reggio nell'Emilia,ITH53,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Reggio Emilia province,Emilia-Romagna region,North-Eastern Italy,Reggio Emilia,6.0,EMSR771,EMSR771,2024-10-21T15:43:26.721092,Estimated population_None_(),150.0,220000.0,NaN,NaN,NaN
4,EMSR771,"Flood in Emilia- Romagna, Italy",Italy has been hit in recent days by a severe ...,Italy|Presidency of the Council of Ministers -...,2024-10-20T12:00:00,2024-10-20T08:24:00,Italy,Modena,7,DEL,0,2024-10-22T16:42:04.899238,Population,number,population_affected_max,NaN,max population,100.0,https://storymaps.arcgis.com/stories/b95e2210e...,Modena,ITH54,intermediate,non-coastal,ITH5,Emilia-Romagna,ITH,Nord-Est,Modena province,Emilia-Romagna region,North-Eastern Italy,Modena,7.0,EMSR771,EMSR771,2024-10-22T16:42:04.899238,Estimated population_None_(),100.0,400000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,EMSR659,"Flood in Emilia-Romagna, Italy","On the 02 May 2023, an intense phase of bad we...",Italy|Presidenza del Consiglio dei Ministri - ...,2023-05-02T18:00:00,2023-05-03T09:39:00,Italy,Spaz

In [103]:
merged_df.to_excel("test_pop.xlsx", index=False)

#### data from population in the new website did not affect the API, in a manual check the new website had also the same data

#### automatic translation did not work